### Forex Arbitrage

&nbsp;

The classic arbitrage in forex trading is triangular arbitrage. For instance, if we wanna short GBP long DKK, we can short GBP long PLN then short PLN long DKK, which may give us more DKK in some rare cases. We can use dijkstra/bellman-ford algorithm to solve the problem. However, high frequency trading is already a major player in forex market. Triangular arbitrage opportunity is like a needle in the haystack. Even if it exists, it would only last for a millisecond or even less. It would have been exploited long before any human knows. Unfortunately, I have never found any triangular arbitrage opportunity throughout my time of trading.

&nbsp;

In [1]:
import os
os.chdir('K:/ecole/github')
import pandas as pd
import numpy as np

#graph adt
#check the below link for more details
# https://github.com/je-suis-tm/graph-theory/blob/master/graph.py
import graph

In [2]:
#forex data can be downloaded in the data subfolder
# https://github.com/je-suis-tm/graph-theory/blob/master/data/forex.csv
df=pd.read_csv('forex.csv')

In [3]:
df

,currency,ask,bid
0,AUDJPY,83.8930,83.8610
1,AUDUSD,0.7599,0.7596
2,CHFJPY,112.0830,112.0370
3,EURCHF,1.1604,1.1601
4,EURGBP,0.8809,0.8806
5,EURJPY,130.0160,129.9680
6,EURUSD,1.1778,1.1773
7,GBPCHF,1.3177,1.3172
8,GBPJPY,147.6490,147.5770
9,GBPUSD,1.3375,1.3367


In [4]:
#the tricky part of forex graph is its computation
#for instance, assume we have EURGBP,GBPUSD
#to get EURUSD, we need to multiply EURGBP by GBPUSD
#however, dijkstra doesnt accept multiplication
#hence, we apply a logarithm transformation
#cuz log(a)+log(b)=log(a*b)
#additionally, some forex rate such as EURGBP is smaller than 1
#after logarithm transformation we get negative numbers(cuz e**0=1)
#we should use negative logarithm transformation to solve it
#the good news is everything kind of clicks through
#since we wanna get an indirect forex rate larger than a direct one
#after negative logarithm transformation
#the objective changes to the smallest transformed rate
#which is consistent with dijkstra!
#to make our life easier, we ignore bid ask spread
forex=graph.graph()

#there is limitation for negative logarithm transformation
#some forex rate such as EURJPY is larger than 100
#after logarithm transformation we end up with positive numbers
#in order to apply dijkstra, we avoid negative weights
for i in df.index:
    if np.log(df['bid'][i])<0:
        forex.append(df['currency'][i][:3],
                     df['currency'][i][3:],-np.log(df['bid'][i]))
    if np.log(1/df['bid'][i])<0:
        forex.append(df['currency'][i][3:],
                     df['currency'][i][:3],-np.log(1/df['bid'][i]))

In [5]:
#this graph adt has excluded negative cycles
forex.reveal()

{'JPY': {'AUD': 4.429160666307817,
  'CHF': 4.718829173882046,
  'EUR': 4.867288266308598,
  'GBP': 4.994350073465887,
  'USD': 4.703892718905732},
 'AUD': {'USD': 0.27496330004400615},
 'USD': {'EUR': 0.16322368110201413,
  'GBP': 0.29020388999529034,
  'CHF': 0.01491061273575424},
 'CHF': {'EUR': 0.14850620829922395, 'GBP': 0.2755082715200721},
 'EUR': {'GBP': 0.12715178566048355},
 'GBP': {}}

![alt text](./preview/forex.png)

### Bellman-Ford Algorithm

&nbsp;

Bellman-Ford algorithm is somewhat similar to Dijkstra's algorithm. Yet, it doesn't require a queue to do Breadth-First-Search traversal. The algorithm iterates through each vertex in the structure multiple times. To be more precise, the number of the iterations equal to the order of the graph structure minus two. Thus, it does not need to keep track of which vertices have been visited. It has one extra round of iteration to detect if there is a negative cycle and raise error if there is one. If there is negative weight, the one last traversal cannot bring convergence. We would end up with a ridiculously small distance caused by several rounds of deduction. Without negative cycle, the algorithm works just like dijkstra to return the optimal steps. With 3 layers of loops, we can easily tell Bellman-Ford has higher time complexity than Dijkstra.

Details of Dijkstra are in the following link

https://github.com/je-suis-tm/graph-theory/blob/master/dijkstra%20shortest%20path.ipynb

&nbsp;

In [6]:
#bellman ford is basically quid pro quo
#it trades higher time complexity for lower space complexity
def bellman_ford(ADT,start,end):
    """Bellman-Ford Algorithm,
    a modified Dijkstra's algorithm to detect negative cycle"""
    
    #distance is a dictionary
    #it keeps track of distance from starting vertex to any vertex
    #before we start any iteration
    #we initialize all distances from start to any vertices to infinity
    #we set the distance[start] to zero
    distance={}
    for i in ADT.vertex():
        distance[i]=float('inf')            
    distance[start]=0  
    
    #pred is a dict as well
    #it keeps track of how we get to the current vertex
    #each time we update distance, we update the predecessor vertex
    #in the end, we can obtain the detailed path from start to end
    pred={}
    
    #dynamic programming
    for _ in range(1,ADT.order()-1):
        for i in ADT.vertex():
            for j in ADT.edge(i):
                try:
                    if distance[i]+ADT.weight(i,j)<distance[j]:
                        distance[j]=distance[i]+ADT.weight(i,j)
                        pred[j]=i
                
                except KeyError:
                    pass
    
    #detect negative cycle
    for k in ADT.vertex():
        for l in ADT.edge(k):
            try:
                assert distance[k]+ADT.weight(k,l)>=distance[l],'negative cycle exists!'
            except KeyError:
                pass
    
    #create the shortest path by backtracking
    #trace the predecessor vertex from end to start
    previous=end
    path=[]
    while pred:
        path.insert(0, previous)
        if previous==start:
            break
        previous=pred[previous]
     
    return distance[end],path

In [7]:
#without negative cycle
#dijkstra vs bellman ford
answer=graph.dijkstra(forex,'USD','EUR')

answer2=bellman_ford(forex,'USD','EUR')

answer==answer2

True

In [8]:
#we revert the answer back to non-logarithm forex rate
np.e**(-answer[0])

0.8494011721736175

In [9]:
#the shortest route is direct route USDEUR
answer[1]

['USD', 'EUR']

In [10]:
#which is consistent with the inverse of EURUSD bid price
1/df['bid'][df['currency']=='EURUSD']

6    0.849401
Name: bid, dtype: float64

In [11]:
#if we try a triangular arbitrage
#USD to JPY to EUR
#we would obtain a smaller number
#it proves that dijkstra delivers the optimal forex rate
step1=float(df['bid'][df['currency']=='EURJPY'])
step2=float(df['bid'][df['currency']=='USDJPY'])
step2/step1

0.8492552012803153

![alt text](./preview/direct.png)

In [12]:
#say there is a glitch in EURCHF bid price
#what will happen?
glitch=df['bid'][df['currency']=='EURCHF'].item()-0.0011
forex.append('CHF','EUR',-np.log(1/glitch))

#reset status for dijkstra
forex.clear(whole=True)

In [13]:
#consistency check
answer=graph.dijkstra(forex,'USD','EUR')

answer2=bellman_ford(forex,'USD','EUR')

answer==answer2

True

In [14]:
#voila, riskless arbitrage
print(f'The triangular arbitrage path is {answer[1]}')

The triangular arbitrage path is ['USD', 'CHF', 'EUR']


![alt text](./preview/arbitrage.png)

In [15]:
#lets build a complete graph adt and implement bellman ford algorithm
forex=graph.graph()
for i in df.index:
    forex.append(df['currency'][i][:3],
                 df['currency'][i][3:],-np.log(df['bid'][i]))
    forex.append(df['currency'][i][3:],
                 df['currency'][i][:3],-np.log(1/df['bid'][i]))

In [16]:
#the complete graph adt
forex.reveal()

{'AUD': {'JPY': -4.429160666307817, 'USD': 0.27496330004400615},
 'JPY': {'AUD': 4.429160666307817,
  'CHF': 4.718829173882046,
  'EUR': 4.867288266308598,
  'GBP': 4.994350073465887,
  'USD': 4.703892718905732},
 'USD': {'AUD': -0.27496330004400615,
  'EUR': 0.16322368110201413,
  'GBP': 0.29020388999529034,
  'CHF': 0.01491061273575424,
  'JPY': -4.703892718905732},
 'CHF': {'JPY': -4.718829173882046,
  'EUR': 0.14850620829922395,
  'GBP': 0.2755082715200721,
  'USD': -0.01491061273575428},
 'EUR': {'CHF': -0.14850620829922395,
  'GBP': 0.12715178566048355,
  'JPY': -4.867288266308598,
  'USD': -0.16322368110201407},
 'GBP': {'EUR': -0.1271517856604836,
  'CHF': -0.2755082715200721,
  'JPY': -4.9943500734658866,
  'USD': -0.2902038899952903}}

In [17]:
#there is a negative cycle
#we got error message
bellman_ford(forex,'USD','EUR')

AssertionError: negative cycle exists!